In [460]:
import sys
import pandas as pd
import numpy as np

# For not truncating output on large ndarrays.
np.set_printoptions(threshold=sys.maxsize)

In [513]:
df = pd.read_csv('401k.csv')

In [514]:
cats = [
  ['<10k', '10-20k', '20-30k', '30-40k', '40-50k', '50-75k', '>75k'],
  ['<30', '30-35', '36-44', '45-54', '55+'],
  ['<12', '12', '13-16', '16+'],
]
_ranges = [
  [
    (0, 9999),
    (10000, 19999),
    (20000, 29999),
    (30000, 39999),
    (40000, 49999),
    (50000, 74999),
    (75000, max(df.inc))
  ],
  [
    (0, 30),
    (30, 35),
    (36, 44),
    (45, 54),
    (55, max(df.age))
  ],
  [
    (0, 11),
    (12, 12),
    (13, 15),
    (16, max(df.educ)),
  ]
]
sources = ['inc', 'age', 'educ']

def generate_dummies(df: pd.DataFrame):
  """
  Generates dummies for each categorical field described in Chernozhukov et al., 2014.
  """
  for _cats, __ranges, source in zip(cats, _ranges, sources):
    for cat, _range in zip(_cats, __ranges):
      df[cat] = (
        (df[source] >= _range[0]) 
        & (df[source] <= _range[1])
      ).astype(int)

generate_dummies(df)

In [515]:
def safe_generate_loginc(df: pd.DataFrame):
  """
  Safe generate loginc. This means that df rows get dropped if income is 
  <= 0 (a case for a few of them).
  """
  df.drop(
    index=df[df.inc <= 0].index,
    inplace=True,
  )
  df['loginc'] = np.log(df.inc)
  assert not df.loginc.isnull().all()

safe_generate_loginc(df)

In [516]:
# List of controls that does not include p401 (intended for interactions).
CONTROLS = ['marr', 'fsize', 'twoearn', 'db', 'pira', 'hown'] + [category for _cats in cats for category in _cats]

In [518]:
def generate_pairwise_interactions(df: pd.DataFrame, controls: list[str]) -> tuple[pd.DataFrame, list[str]]:
  """
  Generate pairwise interactions and insert into dataframe 'df' for every field in controls.

  Returns: DataFrame with all pairwise interactions, plus the list of combined colnames generated.
  """
  pairs = [
    # Generate combinations for all possible pairwise tuples, inclusive of index.
    (controls[i], controls[j]) for i in range(0, len(controls)) for j in range(i, len(controls))
  ]
  combinations_colnames = []
  # Using a separate interaction columns list for convenient merging via pd.concat.
  interaction_columns = []

  for _tuple in pairs:
    colname = f'{_tuple[0]}^2' if _tuple[0] == _tuple[1] else f'{_tuple[0]}:{_tuple[1]}'
    combinations_colnames.append(colname)
    col = (df[_tuple[0]] * df[_tuple[1]]).rename(colname)
    interaction_columns.append(col)

  return (
    pd.concat(
      [df, *interaction_columns],
      axis=1,
    ), 
    combinations_colnames
  )

df_with_interactions, combination_colnames = generate_pairwise_interactions(df, CONTROLS)

In [533]:
from typing import Literal
from sklearn.linear_model import LogisticRegressionCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV

def generate_cv_pipeline_and_fit(
  X: pd.DataFrame, 
  y: pd.Series, 
  reg_type: Literal['lassocv', 'logisticcv'],
  cv: int = 10, 
  random_state: int = 42,
  n_jobs: int = -1,
  tol: int = 1e-2,
) -> list[int]:
  """
  Generates pipeline with requested parameters 'cv', 'random_state', 'n_jobs', and 'tol'. Fits 
  on requested X and y (pd.DataFrame/Series).

  Returns: list of selected covariate indices LassoCV chose in its fitting process.
  """
  if reg_type not in ('lassocv', 'logisticcv'):
    raise NotImplementedError(f'Regression type of {reg_type} not defined.')

  model_type = 1 if reg_type == 'lassocv' else 2
  model = LassoCV(
    cv=cv, 
    random_state=random_state, 
    n_jobs=n_jobs, 
    tol=tol, 
  ) if model_type == 1 else LogisticRegressionCV(
    penalty='l1',
    cv=cv,
    random_state=random_state,
    solver='liblinear',
    tol=tol
  )

  steps = ['scaler', 'lassocv' if model_type == 1 else 'logisticcv']
  pipeline = Pipeline([
    (steps[0], StandardScaler()), 
    (steps[1], model)
  ])

  pipeline.fit(X, y)
  coefs = pipeline[steps[1]].coef_
  selected_indices = np.where(coefs > 1e-6)[0] if model_type == 1 else np.where(coefs[0] > 1e-6)[0]
  return selected_indices

In [551]:
def run_double_lasso_and_obtain_union_of_variables(
  X1: pd.DataFrame, 
  X2: pd.DataFrame, 
  y_i: pd.Series, 
  d_i: pd.Series
) -> list[str]:
  """
  Runs double lasso as outlined by Chernozhukov et al. 2014 for the given parameters X1, X2, y_i and d_i. 

  This is the set of equations:
    y_i = \alpha d_i + x_i'\theta_y + r_{yi} + \epsilon_i 
    d_i = x_i'\theta_d + r_{di} + v_i

  With y_i the outcome of interest and d_i the treatment policy of interest. \theta_d represents the vector
  of coefficients used in prediction of d_i, and \theta_y represents the vector of coefficients used in prediction
  of y_i. We run LASSO on both equations.

  Returns: list of variables obtained by the procedure.
  """
  selected_outcome_indices = generate_cv_pipeline_and_fit(X1, y_i, reg_type='lassocv')
  selected_outcome_variables = X1.columns[selected_outcome_indices]  
  
  selected_treatment_indices = generate_cv_pipeline_and_fit(X2, d_i, reg_type='logisticcv')
  selected_treatment_variables = X2.columns[selected_treatment_indices]

  return list(set(selected_outcome_variables) | set(selected_treatment_variables))

X1 = df_with_interactions[CONTROLS + combination_colnames + ['p401']]
y_i = df.tw

X2 = df_with_interactions[CONTROLS + combination_colnames]
d_i = df.p401

union_of_selected_vars = run_double_lasso_and_obtain_union_of_variables(X1, X2, y_i, d_i)

In [561]:
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')
pd.reset_option('display.float_format')
pd.reset_option('display.max_colwidth')

In [567]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

average_alpha = 0
average_rmse = 0
X3 = df_with_interactions[union_of_selected_vars]
y = df.tw

for i in range(0, 100):
  print(f'{i}...')

  # Generate a 'bootstrapped' alpha coefficient for p401.
  X_train, X_test, y_train, y_test = train_test_split(
    X3, 
    y,
    test_size=0.2,
  )
  X_train = sm.add_constant(X_train)
  X_test = sm.add_constant(X_test)

  # Generate metrics of interest.
  res = sm.OLS(y_train, X_train).fit()
  y_pred = res.predict(X_test)
  rmse = np.sqrt(((y_test - y_pred) ** 2).mean())

  average_alpha = (average_alpha * i + res.params['p401']) / (i + 1)
  average_rmse = (average_rmse * i + rmse) / (i + 1)

print(f'Average alpha: {average_alpha}')
print(f'Average root mean squared error: {average_rmse}')

0...
1...
2...
3...
4...
5...
6...
7...
8...
9...
10...
11...
12...
13...
14...
15...
16...
17...
18...
19...
20...
21...
22...
23...
24...
25...
26...
27...
28...
29...
30...
31...
32...
33...
34...
35...
36...
37...
38...
39...
40...
41...
42...
43...
44...
45...
46...
47...
48...
49...
50...
51...
52...
53...
54...
55...
56...
57...
58...
59...
60...
61...
62...
63...
64...
65...
66...
67...
68...
69...
70...
71...
72...
73...
74...
75...
76...
77...
78...
79...
80...
81...
82...
83...
84...
85...
86...
87...
88...
89...
90...
91...
92...
93...
94...
95...
96...
97...
98...
99...
Average alpha: 11713.459820905493
Average root mean squared error: 85824.28082773615
